# Cortisol Plots

In [ ]:
from pathlib import Path
import json

import pandas as pd
import numpy as np
import pingouin as pg

import biopsykit as bp
from biopsykit.stats import StatsPipeline
from biopsykit.io import load_long_format_csv
from biopsykit.utils.dataframe_handling import multi_xs

from fau_colors import cmaps, register_fausans_font

import matplotlib.pyplot as plt
import seaborn as sns

from carwatch_analysis.io import load_cortisol_samples_reporting_times
from carwatch_analysis.utils import describe_groups_df
from carwatch_analysis.stats import create_unique_night_id
from carwatch_analysis.data_processing.unit_digits import compute_time_unit_digits
from carwatch_analysis.plotting import multi_paired_plot_auc, paired_plot_auc, time_unit_digits_histogram


%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
register_fausans_font()
plt.close("all")

palette = sns.color_palette(cmaps.faculties)

theme_kwargs = {"context": "talk", "style": "ticks", "palette": palette, "font_scale": 1.25}
sns.set_theme(**theme_kwargs)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "FAUSans Office"

pg.options["round"] = 4

palette

## Setup Paths

In [ ]:
base_path = Path("../..")
export_path = base_path.joinpath("exports")
result_path = base_path.joinpath("results")
stats_path = result_path.joinpath("statistics")

img_path = result_path.joinpath("plots/graphical_abstract")

paper_path = Path(json.load(Path("../paper_path.json").open(encoding="utf-8"))["paper_path"])
paper_img_path = paper_path.joinpath("img/graphical_abstract")

bp.utils.file_handling.mkdirs([result_path, stats_path, img_path, paper_img_path])

In [ ]:
reporting_types = ["AW & ST: Selfreport", "AW & ST: App"]
reporting_types_new = ["Selfreport", "App"]

## Cortisol Features

### Load Data

In [ ]:
cort_features = load_long_format_csv(export_path.joinpath("cortisol_features_processed_all_reporting_types.csv"))
cort_features = create_unique_night_id(cort_features)
cort_features.head()

### Data Selection

In [ ]:
cort_features_analysis = cort_features.copy()
cort_features_analysis = cort_features_analysis.reindex(reporting_types, level="reporting_type")
cort_features_analysis = cort_features_analysis.rename(lambda x: x.split(" ")[-1], level="reporting_type")
cort_features_analysis = cort_features_analysis.xs("auc_i", level="saliva_feature", drop_level=False)
cort_features_analysis.head()

### Plots

#### $AUC_I$

In [ ]:
reporting_types = ["Selfreport", "App"]

fig, ax = paired_plot_auc(
    data=cort_features_analysis, saliva_feature="auc_i", reporting_types=reporting_types, figsize=(6, 4)
)

ax.set_ylabel(None)
fig.suptitle(None)
ax.set_title("$AUC_I$")
fig.tight_layout(pad=0)

for path in [img_path, paper_img_path]:
    bp.utils.file_handling.export_figure(fig, "img_pairedplot_auci_abstract", path, ["pdf", "png"], dpi=300)

## Time Unit Information

### Load Data

In [ ]:
cort_path = export_path.joinpath("cortisol_samples_processed_all_reporting_types.csv")
cort_samples = load_cortisol_samples_reporting_times(cort_path)
# rename condition
cort_samples.head()

### Data Preparation

In [ ]:
time_unit_digits = compute_time_unit_digits(cort_samples)

time_unit_digits = time_unit_digits.reindex(reporting_types, level="reporting_type")
time_unit_digits = time_unit_digits.rename(lambda x: x.split(" ")[-1], level="reporting_type")
time_unit_digits.head()

### Histogram Plot

In [ ]:
paper_img_path.resolve()

In [ ]:
ylim = [0, 35]

fig, axs = time_unit_digits_histogram(
    data=time_unit_digits,
    x="sample_minute",
    reporting_type_order=reporting_types_new,
    suptitle="Sampling Times",
    ylim=ylim,
    figsize=(8, 4),
    column_title=False
)


for path in [img_path, paper_img_path]:
    bp.utils.file_handling.export_figure(fig, "img_unit_digits_sampling_time_graphical_Abstract", path, ["pdf", "png"], dpi=300)